In [1]:
import gzip
import torch
import torchvision
import numpy as np 

import idx2numpy
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [2]:
def load_one_dataset(path):
    
    f = gzip.open(path, 'rb')
    data = torch.from_numpy(idx2numpy.convert_from_file(f))
    f.close()
    
    return(data)


def load_all_datasets(train_imgs, train_labs, test_imgs, test_labs, batch_size):
    
    
    train_images = load_one_dataset(train_imgs).type(torch.float32)
    train_labels = load_one_dataset(train_labs).type(torch.long)
    train = list(zip(train_images, train_labels))
    
    test_images = load_one_dataset(test_imgs).type(torch.float32)
    test_labels = load_one_dataset(test_labs).type(torch.long)
    test = list(zip(test_images, test_labels))
    
    train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True, num_workers=2)
    test_loader = torch.utils.data.DataLoader(test, batch_size=batch_size, shuffle=False, num_workers=2)
    
    return(train_loader, test_loader)

In [3]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 256)
        self.fc2 = nn.Linear(256, 64)
        self.fc3 = nn.Linear(64, 10)
        
    def forward(self, x):
        x = x.view(-1, 28*28)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        return x

In [4]:
def train(epochs=2, lr=0.001, momentum=0.9, batch_size=512):
    
    
    # Set paths to datasets
    paths = {
        
        'train_imgs': 'train-images-idx3-ubyte.gz',
        'train_labs': 'train-labels-idx1-ubyte.gz',
        'test_imgs': 't10k-images-idx3-ubyte.gz',
        'test_labs': 't10k-labels-idx1-ubyte.gz'
    }
    
    # Load datasets
    train_loader, test_loader = load_all_datasets(**paths, batch_size = 256)
    
    # Set parameters
    net = Net()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=lr, momentum=momentum)
    
    # Loop over the dataset multiple times
    for epoch in range(epochs):  
        
        # Initialize running loss
        running_loss = 0.0
        
        # Iterate through data now
        for i, data in enumerate(train_loader):
            
            # Get the inputs: data is a list of [inputs, labels]
            inputs, labels = data

            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward
            outputs = net(inputs)
            
            # Calculate loss
            loss = criterion(outputs, labels)
            
            # Backward
            loss.backward()
            
            # Optimize
            optimizer.step()

            # Print statistics
            running_loss += loss.item()
            
            # Print every 100 mini-batches
            if i % 100 == 0:    
                print('[%d, %5d] loss: %.3f' % (epoch + 1, running_loss / 100))
                running_loss = 0.0
    
    # Print message
    print('Done training...')

In [ ]:
train(epochs=100)